In [1]:
import sys, os

SRC_PATH = os.path.abspath(os.path.join("..", "src"))
sys.path.insert(0, SRC_PATH)

print("SRC_PATH:", SRC_PATH)
print("Exists:", os.path.exists(SRC_PATH))


SRC_PATH: c:\Users\ASUS\Desktop\Text-Summarizer-Project-\src
Exists: True


In [2]:
print(os.listdir(SRC_PATH))
print(os.listdir(os.path.join(SRC_PATH, "textSummarizer")))
print(os.listdir(os.path.join(SRC_PATH, "textSummarizer", "components")))


['textSummarizer', 'textSummarizer.egg-info']
['components', 'config', 'constants', 'entity', 'logging', 'pipeline', 'utils', '__init__.py', '__pycache__']
['data_ingestion.py', 'data_transformation.py', 'data_validation.py', 'model_evaluation.py', 'model_trainer.py', '__init__.py', '__pycache__']


In [3]:
from textSummarizer.components.model_evaluation import ModelEvaluation
from textSummarizer.config.configuration import ConfigurationManager


c:\Users\ASUS\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import sys

# 1️⃣ Set working directory to project root
os.chdir(r"C:\Users\ASUS\Desktop\Text-Summarizer-Project-")

# 2️⃣ Add src folder to sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

print("Current working directory:", os.getcwd())
print("sys.path includes:", sys.path[-3:])  # check last 3 paths


Current working directory: C:\Users\ASUS\Desktop\Text-Summarizer-Project-
sys.path includes: ['c:\\Users\\ASUS\\anaconda3\\envs\\textS\\lib\\site-packages\\win32\\lib', 'c:\\Users\\ASUS\\anaconda3\\envs\\textS\\lib\\site-packages\\Pythonwin', 'C:\\Users\\ASUS\\Desktop\\Text-Summarizer-Project-\\src']


In [5]:
import sys, os
sys.path.insert(0, os.path.abspath("../src"))


In [6]:
import os

In [7]:
%pwd

'C:\\Users\\ASUS\\Desktop\\Text-Summarizer-Project-'

In [8]:
os.chdir("../")

In [9]:
%pwd

'C:\\Users\\ASUS\\Desktop'

In [10]:
import os

# Set your working directory to the project root
os.chdir(r"C:\Users\ASUS\Desktop\Text-Summarizer-Project-")
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\ASUS\Desktop\Text-Summarizer-Project-


In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [12]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [13]:
import os

# Force working directory to project root
os.chdir(r"C:\Users\ASUS\Desktop\Text-Summarizer-Project-")

print("Current working directory:", os.getcwd())
print("Config exists:", os.path.exists("config/config.yaml"))
print("Params exists:", os.path.exists("params.yaml"))


Current working directory: C:\Users\ASUS\Desktop\Text-Summarizer-Project-
Config exists: True
Params exists: True


In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config


In [15]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [16]:
import evaluate
import torch



In [17]:
class ModelEvaluation:
    def __init__(self, config):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = evaluate.load("rouge")


        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)

        



In [18]:
def evaluate(self):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
    model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
        self.config.model_path
    ).to(device)

    dataset_samsum_pt = load_from_disk(self.config.data_path)

    # ✅ fixed line
    rouge_metric = evaluate.load("rouge")

    score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt["test"][:10],
        rouge_metric,
        model_pegasus,
        tokenizer,
        batch_size=2,
        column_text="dialogue",
        column_summary="summary",
    )

    rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    rouge_dict = {rn: score[rn].mid.fmeasure for rn in rouge_names}

    df = pd.DataFrame(rouge_dict, index=["pegasus"])
    df.to_csv(self.config.metric_file_name, index=False)


In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm


class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """
        Split the dataset into smaller batches
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(
        self,
        dataset,
        metric,
        model,
        tokenizer,
        batch_size=16,
        device=None,
        column_text="article",
        column_summary="highlights",
    ):
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"

        article_batches = list(
            self.generate_batch_sized_chunks(dataset[column_text], batch_size)
        )
        target_batches = list(
            self.generate_batch_sized_chunks(dataset[column_summary], batch_size)
        )

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches),
            total=len(article_batches),
        ):
            inputs = tokenizer(
                article_batch,
                max_length=1024,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                length_penalty=0.8,
                num_beams=8,
                max_length=128,
            )

            decoded_summaries = [
                tokenizer.decode(
                    s,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=True,
                )
                for s in summaries
            ]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(
                predictions=decoded_summaries,
                references=target_batch,
            )

        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_path
        ).to(device)

        # Load processed dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        # ✅ NEW API (correct)
        rouge_metric = evaluate.load("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt["test"][:10],
            rouge_metric,
            model_pegasus,
            tokenizer,
            batch_size=2,
            column_text="dialogue",
            column_summary="summary",
        )

        # ✅ FIXED: evaluate returns floats directly
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index=["pegasus"])
        df.to_csv(self.config.metric_file_name, index=False)

        print("\nROUGE Evaluation Scores:")
        print(df)


In [20]:
from textSummarizer.components.model_evaluation import ModelEvaluation
from textSummarizer.config.configuration import ConfigurationManager

config = ConfigurationManager()
eval_config = config.get_model_evaluation_config()

evaluator = ModelEvaluation(config=eval_config)
evaluator.evaluate()


[2026-01-04 15:17:00,005: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-04 15:17:00,010: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-04 15:17:00,010: INFO: common: created directory at: artifacts]
[2026-01-04 15:17:00,015: INFO: common: created directory at: artifacts/model_evaluation]


100%|██████████| 5/5 [04:39<00:00, 55.91s/it]

[2026-01-04 15:21:52,626: INFO: rouge_scorer: Using default tokenizer.]
